# Preprocess Data

In [20]:
path_to_feature_dir = "safety/features/"
path_to_label_dir = "safety/labels/"

In [4]:
import os
if not os.path.isdir("safety"):
  !wget https://s3-ap-southeast-1.amazonaws.com/grab-aiforsea-dataset/safety.zip && unzip safety && rm safety.zip
else:
  print("Data already downloaded")

Data already downloaded


In [5]:
import h5py
import numpy as np 
import pandas as pd 
from scipy import stats
from sklearn import preprocessing
import gc
print(os.listdir("safety"))
from sklearn.externals import joblib
from tqdm import tqdm_notebook as tqdm

['.DS_Store', 'labels', 'data_dictionary.xlsx', 'features']


In [6]:
def read_multiple_csv(path):
    files = os.listdir(path)
    df = None
    pbar = tqdm(total=len(files))
    for f in files:
        ext = f.split(".")
        if len(ext)>0 and ext[-1] == "csv":
            #print(f)
            _ = pd.read_csv(os.path.join(path, f))
            if df is None:
                df = _
            else:
                df = df.append(_)
        pbar.update(1)
    del _
    gc.collect()
    return df

In [9]:
%%time
feature_df = read_multiple_csv(path_to_feature_dir)

CPU times: user 36.2 s, sys: 6.19 s, total: 42.4 s
Wall time: 42.4 s


In [4]:
#feature_df.to_parquet("data/original_features.parquet")

In [5]:
feature_df.sort_values(by='bookingID').head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
494610,0,8.0,316.671051,-1.678543,-9.668259,-1.181485,0.005701,-0.262353,-0.050388,948.0,2.761940
853220,0,16.0,142.124496,-0.642108,-9.626239,-1.777237,-0.029558,-0.076531,0.045855,174.0,0.227891
833328,0,8.0,342.453766,-1.219467,-9.119461,-1.399210,0.004682,0.025333,0.050477,1396.0,18.803959
27526,0,16.0,238.402939,-0.462814,-9.436029,-1.591064,0.032511,0.009371,-0.010461,259.0,2.362042
344962,0,8.0,1.363404,-1.917352,-9.133069,-0.564648,0.135485,0.086545,-0.011729,1575.0,14.235041


In [10]:
# derive acceleration from speed
feature_df = feature_df.fillna(0)
feature_df['acceleration'] = (feature_df['Speed']-feature_df['Speed'].shift())/\
(feature_df['second']-feature_df['second'].shift())
feature_df = feature_df.fillna(0)
feature_df['acceleration']  = feature_df['acceleration'].replace([np.inf, -np.inf], 0)

In [11]:
## check total NaN
feature_df.isnull().T.any().T.sum()

0

In [12]:
%%time
# remove outlier rows
cols = ['Bearing','acceleration_x', 'acceleration_y', 'acceleration_z','gyro_x', 'gyro_y', 'gyro_z', 'acceleration', 'Speed']
pbar = tqdm(total=len(cols))
for i in cols:
    feature_df['z_'+i] = np.abs(stats.zscore(feature_df[i]))
    feature_df = feature_df[feature_df['z_'+i] <= 3]
    feature_df = feature_df.drop(columns=['z_'+i])
    pbar.update(1)

CPU times: user 21.7 s, sys: 28.5 s, total: 50.2 s
Wall time: 50.2 s


In [13]:
# remove zero values
pbar = tqdm(total=len(cols))
for i in cols:
    feature_df = feature_df[feature_df[i] != 0 ]
    pbar.update(1)

In [14]:
# scale the data
scaler_filename = "models/min_max_scaler.save"
min_max_scaler = joblib.load(scaler_filename) 
x_scaled = min_max_scaler.transform(feature_df.iloc[:,1:])
cols = feature_df.iloc[:,1:].columns
feature_df.loc[:,cols] = x_scaled
del x_scaled
gc.collect()

0

In [15]:
# 12189032
len(feature_df)

12189032

In [16]:
d = feature_df.iloc[:,1:].describe()

In [17]:
d

,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,acceleration
count,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07,1.218903e+07
mean,1.868821e-03,5.002001e-01,4.980047e-01,4.836508e-01,4.931750e-01,4.996080e-01,5.009444e-01,5.003409e-01,7.540733e-02,3.634180e-01,5.000743e-01
std,1.472615e-02,2.810122e-01,1.249095e-01,1.773207e-01,1.559571e-01,1.098413e-01,6.941128e-02,1.233218e-01,5.552369e-02,2.177102e-01,4.365173e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.196091e-04,2.583335e-01,4.288701e-01,2.311074e-01,3.973193e-01,4.592295e-01,4.786042e-01,4.493230e-01,3.153153e-02,1.676628e-01,4.943473e-01
50%,7.010598e-04,4.972228e-01,4.977782e-01,5.936837e-01,4.871374e-01,5.004978e-01,5.012272e-01,5.006580e-01,6.659362e-02,3.454537e-01,4.998480e-01
75%,1.360881e-03,7.444454e-01,5.678481e-01,6.091605e-01,5.879140e-01,5.399913e-01,5.242055e-01,5.515520e-01,1.081081e-01,5.372653e-01,5.059982e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [18]:
# padding value
pad_value = 2.0

In [19]:
# checking if any of column having values equal or less than pad_value
for col in d.columns:
    if d[col]['min'] >= pad_value:
        print(col, d[col]['min'])

In [21]:
label_df = read_multiple_csv(path_to_label_dir)
label_df.head()

,bookingID,label
0,111669149733,0
1,335007449205,1
2,171798691856,0
3,1520418422900,0
4,798863917116,0


In [22]:
print(len(label_df))
# we can't keep both due to label contains both 0 and 1
label_df = label_df.drop_duplicates(subset=['bookingID'], keep=False)
print(len(label_df))

20018
19982


In [17]:
label_df = feature_df.groupby('bookingID',as_index=False)\
.agg({'Accuracy':'count'})\
.sort_values(by='Accuracy', ascending=False)\
.rename(columns={'Accuracy':'count'})\
.merge(label_df, on='bookingID', how='left')

label_df.head()

,bookingID,count,label
0,1374389534819,4361,1.0
1,1108101562533,3384,1.0
2,1365799600208,3014,1.0
3,412316860458,2925,1.0
4,283467841668,2831,1.0


In [18]:
null_booking_ids = label_df[label_df['label'].isnull()]['bookingID']

In [21]:
len(null_booking_ids)

18

In [22]:
feature_df = feature_df[~feature_df['bookingID'].isin(null_booking_ids)]

In [23]:
max_row = label_df['count'].max()
print("Maximum row foreach bookingID",max_row)

Maximum row foreach bookingID 4361


In [24]:
label_df = label_df.dropna().sort_values(by='bookingID')

In [25]:
# create padding
booking_ids = label_df['bookingID'].values
N = len(booking_ids)
print("Total booking ID",N)
dimension = len(feature_df.columns)-1
print("dimension", dimension)
max_seq_len = 1000
Xpad = np.full((N,max_seq_len, dimension), fill_value=pad_value)
print(Xpad.shape)

Total booking ID 19863
dimension 11
(19863, 1000, 11)


In [26]:
booking_ids

array([            0,             1,             2, ..., 1709396983966,
       1709396983971, 1709396983975])

In [27]:
%%time
## TODO looking for faster way to perform this operation
i = 0
count = {'truncate':0, 'pad':0}
pbar = tqdm(total=N)
for booking_id in booking_ids:
    matricData = feature_df[feature_df['bookingID'] == booking_id].iloc[:,1:].values
    #print(matricData.shape)
    if matricData.shape[0] <= 0 :
      print(booking_id, i)
    if matricData.shape[0] >= max_seq_len:
      # truncate
      # take at center
      center = int(matricData.shape[0] / 2)
      begin = center - int(max_seq_len/2) #including
      end = center + int(max_seq_len/2) #excluding
      Xpad[i, :, :] = matricData[begin:end, :]
      count['truncate'] += 1
    else:
      # padding
      #print(matricData.shape)
      Xpad[i, :matricData.shape[0], :] = matricData
      count['pad'] += 1
    i += 1
    del matricData
    gc.collect()
    pbar.update(1)

CPU times: user 39min 30s, sys: 6.48 s, total: 39min 37s
Wall time: 39min 24s


In [28]:
count

{'pad': 16717, 'truncate': 3146}

In [29]:
Xpad[1]

array([[5.36104582e-04, 1.49999939e-01, 5.26561009e-01, ...,
        1.10177745e-01, 6.30369634e-01, 5.08985616e-01],
       [5.36104582e-04, 7.49998060e-02, 4.83751880e-01, ...,
        7.93766740e-02, 3.72639238e-01, 4.89508203e-01],
       [5.36104582e-04, 6.13889647e-01, 5.97610307e-01, ...,
        4.55320185e-02, 2.01356957e-01, 4.95956972e-01],
       ...,
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00]])

In [30]:
%%time
with h5py.File('data/Xpad.h5', 'w') as hf:
    hf.create_dataset("Xpad",  data=Xpad)

CPU times: user 32.3 ms, sys: 2.12 s, total: 2.15 s
Wall time: 15.5 s


In [31]:
print(len(label_df))

19863


In [32]:
label_df.to_parquet("data/bookingID.parquet")

In [ ]:
# read back
with h5py.File('Xpad.h5', 'r') as hf:
    data = hf['Xpad'][:]

In [22]:
#np.save('Xpad.npy', Xpad)
#np.save('booking_ids.npy', np.array(booking_ids))
#new_num_arr = np.load('data.npy') # load

In [80]:
np.argwhere(Xpad == 0)

array([[    0,   383,     8],
       [    1,   542,     8],
       [    2,    70,     8],
       ...,
       [19875,   184,     8],
       [19877,    64,     8],
       [19880,   368,     8]])

In [79]:
# check for any NaN
np.argwhere(np.isnan(Xpad))

array([], shape=(0, 3), dtype=int64)